In [1]:
import json
import os
import numpy as np
import pandas as pd

In [2]:
train = pd.read_json('train.json', typ = 'frame')

In [3]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [4]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [5]:
# 태그 별 매핑 빈도 수 저장 
tag_cnt = plylst_tag_map.groupby('tags').tags.count().reset_index(name = 'mapping_cnt')
tag_cnt['tags'] = tag_cnt['tags'].astype(str)
tag_cnt['mapping_cnt'] = tag_cnt['mapping_cnt'].astype(int)

# 빈도 수가 10회 이상인 태그만 저장
small_tag = tag_cnt[tag_cnt['mapping_cnt'] >= 3]
word_count = list(zip(tag_cnt['tags'], tag_cnt['mapping_cnt']))
print(len(small_tag))
print(len(tag_cnt))
taglist = list(small_tag['tags'])

7785
29160


In [6]:
tagdf=pd.DataFrame(data = small_tag, columns=['tags'])
print(tagdf)

           tags
1            00
2           007
4           00s
5           00년
6          00년대
...         ...
29140      힙합추천
29142      힙합클럽
29146      힙합트랙
29150  힙합플레이리스트
29155        힙해

[7785 rows x 1 columns]


In [7]:
tagdf['song']=np.nan

In [8]:
array =tagdf.to_numpy()
print(array)
for i in range(len(small_tag)):
    array[i,1]=[]
print(array)

[['00' nan]
 ['007' nan]
 ['00s' nan]
 ...
 ['힙합트랙' nan]
 ['힙합플레이리스트' nan]
 ['힙해' nan]]
[['00' list([])]
 ['007' list([])]
 ['00s' list([])]
 ...
 ['힙합트랙' list([])]
 ['힙합플레이리스트' list([])]
 ['힙해' list([])]]


In [12]:
# 플레이리스트 태그(tags)와 수록곡(songs) 추출
tag_song = train[['tags', 'songs']]
first_tag_song=train[['tags', 'songs']][0:1000]

In [10]:
first_tag_song

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."
...,...,...
4995,"[산책, 기분전환, 여행]","[360221, 207723, 628929, 211675, 99326, 452979..."
4996,"[휴식, 매장음악, 가을, 카페, 매장, 드라이브, 재즈, 집, 감성, 기분전환]","[20303, 74567, 667990, 310600, 24762, 185244, ..."
4997,"[해피송, 카페, 사랑노래, 러브, 드라이브, 커플, 연애, 설렘, 사랑, 러브송]","[234600, 255158, 17562, 470573, 316600, 474687..."
4998,"[힐링, 휴식, 설렘, 사랑]","[532114, 264553, 383960, 282365, 117799, 20731..."


In [40]:
first_tag_song[0:1]

,tags,songs
100,"[휴식, 힐링, 기분전환]","[701995, 167631, 493651, 535146, 415862, 11375..."


In [ ]:
for i in range(0,1000):
    for j in range(len(taglist)):
        if taglist[j] in first_tag_song['tags'][i]:
            array[j][1]=list(set(array[j][1] + first_tag_song['songs'][i]))

In [32]:
new = pd.DataFrame(data=array, columns=['tag','song'])]

SyntaxError: invalid syntax (<ipython-input-32-7682a7e00f4a>, line 1)

In [34]:
new.to_json('tagsong4.json',orient='records')

In [45]:
new